<img src="../images/logo.jpg" style="width:85px;height:85px;float:left" /><h1 style="position:relative;float:left;display:inline">DCGAN in Theory and Practice</h1>

<a href="https://colab.research.google.com/github/zurutech/gans-from-theory-to-production/blob/master/3.%20TFGAN/3.1.%20DCGAN%20in%20Theory%20and%20Practice.ipynb">
<img align="left" src="https://cdn-images-1.medium.com/max/800/1*ZpNn76K98snC9vDiIJ6Ldw.jpeg"></img>
</a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Theory" data-toc-modified-id="Theory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Theory</a></span><ul class="toc-item"><li><span><a href="#Enter-DCGAN" data-toc-modified-id="Enter-DCGAN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Enter DCGAN</a></span></li><li><span><a href="#Generator:-from-noise-to-insight" data-toc-modified-id="Generator:-from-noise-to-insight-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generator: from noise to insight</a></span><ul class="toc-item"><li><span><a href="#Deconvolution" data-toc-modified-id="Deconvolution-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Deconvolution</a></span></li><li><span><a href="#Batch-Normalization" data-toc-modified-id="Batch-Normalization-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Batch Normalization</a></span></li><li><span><a href="#generator_fn()" data-toc-modified-id="generator_fn()-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>generator_fn()</a></span></li></ul></li><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Discriminator</a></span><ul class="toc-item"><li><span><a href="#discriminator_fn()" data-toc-modified-id="discriminator_fn()-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>discriminator_fn()</a></span></li></ul></li><li><span><a href="#Loss-function:-a-bridge-between-two-networks" data-toc-modified-id="Loss-function:-a-bridge-between-two-networks-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Loss function: a bridge between two networks</a></span></li></ul></li><li><span><a href="#Practice---Introduction" data-toc-modified-id="Practice---Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Practice - Introduction</a></span><ul class="toc-item"><li><span><a href="#Input-Functions" data-toc-modified-id="Input-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Input Functions</a></span><ul class="toc-item"><li><span><a href="#Loading-Images" data-toc-modified-id="Loading-Images-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Loading Images</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#TensorBoard" data-toc-modified-id="TensorBoard-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>TensorBoard</a></span></li></ul></li><li><span><a href="#Predictions" data-toc-modified-id="Predictions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Predictions</a></span><ul class="toc-item"><li><span><a href="#predict_input_fn()" data-toc-modified-id="predict_input_fn()-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>predict_input_fn()</a></span></li></ul></li><li><span><a href="#Preparing-for-Production" data-toc-modified-id="Preparing-for-Production-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preparing for Production</a></span><ul class="toc-item"><li><span><a href="#serving_input_receiver_fn" data-toc-modified-id="serving_input_receiver_fn-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>serving_input_receiver_fn</a></span></li><li><span><a href="#Exporting-the-model-for-production" data-toc-modified-id="Exporting-the-model-for-production-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Exporting the model for production</a></span></li><li><span><a href="#Local-CloudML-Predictions-testing" data-toc-modified-id="Local-CloudML-Predictions-testing-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Local CloudML Predictions testing</a></span><ul class="toc-item"><li><span><a href="#Generate-the-test-noise" data-toc-modified-id="Generate-the-test-noise-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Generate the test noise</a></span></li><li><span><a href="#Execute-the-test" data-toc-modified-id="Execute-the-test-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Execute the test</a></span></li></ul></li></ul></li><li><span><a href="#To-the-serving" data-toc-modified-id="To-the-serving-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>To the serving</a></span></li><li><span><a href="#NOTES" data-toc-modified-id="NOTES-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>NOTES</a></span></li><li><span><a href="#Links" data-toc-modified-id="Links-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Links</a></span></li></ul></div>

In [ ]:
! pip install tensorflow==1.13.1
#! pip install tensorflow-gpu==1.13.1

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import UpSampling2D, Conv2D, BatchNormalization
from tensorflow.keras.layers import Reshape, Activation, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

# Import tfgan from contrib
tfgan = tf.contrib.gan

## Theory

### Enter DCGAN

![DCGAN](images/dcgan.png)

If you take a look at an [impressive list of GANs](), you see that DCGAN, the architecture of our choice, is just a small drop in the ocean. While there may be sexier ones, very few offers the same package of clarity, stability, performances and computational efficiency, these are the main one reason why DCGAN is considered one of the cornerstones of this field.

Proposed in a 2015 paper by Alec Radford, Luke Metz, Soumith Chintala, the idea behind DCGAN is quite straightforward: combining a set of architectural constraints with the power of CNN yields us a robust, stable, and competitive model. Succeeding where many others failed, the authors present us with a simple model, asymmetrical architecture, 4 deconvolutional layers at the Generator with 4 layers of convolution at the Discriminator with the following constraints:

- All pooling layers are replaced with stranded convolutions (discriminator) and fractional-stride convolutions (generator).
- Batch-normalization used in both networks.
- Removal of fully connected layers (except for the discriminator output)
- `ReLU` for all Generator layers except the output, which uses `tanh`.
- `LeakyReLU` activation in the discriminator for all layers.

Although there has been very recent advancements in state of the art (i.e., CoordConv, Spectral Normalization), we felt that before venturing into the bleediest of the edges it is essential to have a firm understanding of the basic concept. We leave you (an opinionated) list of further resources you can use to get up to speed to the most exciting line of research.

### Generator: from noise to insight

![DCGAN Generator](images/dcgan_generator.png)

If you recall the theoretical explanation, the Generator is the network responsible for the data-generation, learning how to fool its discriminator allows it to produce realistic results "sampled' from the learned manifold.

The most common type of generator_inputs is pure and straightforward noise; however, more specialized GANs may require additional parameters. Since our full-demo uses a Deep Convolutional GAN (DCGAN), we don't need any other parameters.

#### Deconvolution

We do not enter int the theoretical depth of the de-convolution operation, we just give the intuitve idea behind this operation.

> When we have neural networks generate images, we often have them build them up from low resolution, high-level 
> descriptions. This allows the network to describe the rough image and then fill in the details.
>
> In order to do this, we need some way to go from a lower resolution image to a higher one. We generally do this with the deconvolution operation. 
> Roughly, deconvolution layers allow the model to use every point in the small image to “paint” a square in the larger one.

In practice, the deconvolution operation is often implemented by resizing usin bilinear or nearest neighbor interpolation followed by a convolution operation.

#### Batch Normalization

We do not enter into the theoretical depth of `BatchNormalization`; what you need to know is that TensorFlow makes it very easy to implement such an operation.

```python
tf.keras.layers.BatchNormalization()
```

#### generator_fn()

In [ ]:
def get_generator():
    """ Create a model that takes in a matrix of random values as input and outputs images """
    model = Sequential()
    model.add(Dense(input_dim=100, units=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*8*8))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((8, 8, 128), input_shape=(128*8*8,))) # 8x8 image
    model.add(UpSampling2D(size=(2, 2))) # 16x16 image
    model.add(Conv2D(128, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2))) # 32x32 image
    model.add(Conv2D(128, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2))) # 64x64 image
    model.add(Conv2D(3, (5, 5), padding='same'))
    model.add(Activation('tanh'))

    return model



def generator_fn(inputs, mode):
    """Generator producing images from noise.

        Args:
            noise: A single Tensor representing noise.
            mode: tf.estimator.ModeKeys

        Returns:
            A 64x64 (None, 4096) flattened tensor whose values are
            inside the (-1, 1) range."""
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    model = get_generator()
    out = model(inputs, is_training)
    return out

### Discriminator

![DCGAN Discriminator](images/dcgan_discriminator.png)

As for the `input_fn()` we can reuse all the code we defined for the vanilla `Estimator` describing the discriminator network architecture.

#### discriminator_fn()

In [ ]:
def get_discriminator():
    model = Sequential()
    model.add(
        Conv2D(
            64,
            (5, 5),
            padding='same',
            input_shape=(64, 64, 3)
            )
        )
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))

    return model

def discriminator_fn(inputs, conditioning, mode):
    """Build the Discriminator network.
    Args:
        features: a batch of images to classify, expected input shape (None, 64, 64, 3)
        conditioning: a batch of labels, it is used for conditioning in the some model (es Conditional GAN).
            GANEstimator wants this parameters around, just define an arguments so that discriminator_fn is not broken.
        mode: tf.estimator.ModeKey
    
    Returns:
            The output (logits) of the discriminator.
    """
    
    # In every mode, define the model
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    model = get_discriminator()
    out = model(inputs, is_training)
    print("D in", inputs)
    print("D out", out)
    return out

### Loss function: a bridge between two networks

There are a lot of loss functions usable in GANs'architectures from very domain-specific ones to ones that are perfect for general use cases. Since our goal here falls in the latter category, we use the so-called **Non-Saturating Loss** which is the non-saturating variant of the **MinMax Loss** proposed by Goodfellow in the [original paper]().

As stated earlier, one of the beauties of `TFGAN` is its offerings losses ready for use, if you cannot find the loss you want, you can create your own.

We use the following two losses:

```python
generator_loss_fn=tfgan.losses.minimax_generator_loss

discriminator_loss_fn=tfgan.losses.minimax_discriminator_loss
```

It is that easy.

## Practice - Introduction

Having seen the general structure of the TFGAN library, we dive right into the model architecture.

In our showcasing of the TensorFlow API, we have built an image recognition network, GANs, however, require the training of both a Generator and a Discriminator together. While this task is either verbose and possibly performance-constrained (in the case of writing it in vanilla TensorFlow) or complicated (Estimator API) thanks to GANEstimator it becomes remarkably simple.

### Input Functions

The input function for the `GANEstimator` is very much the same as the one we would define for a normal `tf.Estimator`.


>    The function should construct and return one of the following:
>
>    A tf.data.Dataset object: Outputs of Dataset object must be a tuple (features, labels) with same constraints as below.
>
>    A tuple (features, labels): Where `features` is a Tensor or a dictionary of string feature name to Tensor and `labels` is a Tensor or a dictionary of string label name to Tensor. Model_fn consumes both features and labels. They should satisfy the expectation of model_fn from inputs.


Thus, we just have to follow the GANEstimator `input_fn` definition requirements. We have to:

1. Structure the code in the same way the `GANEstimator` wants **function objects** as inputs thus the need to wrap them. 
2. Return **noise** first and then the **real_data** (here called `images`) and optionally **labels**.

#### Loading Images

We use the Celeb-A dataset downloaded using tensorflow-datasets (tfds) to easily create the input function that satisfies all the requirements.

In [ ]:
import tensorflow_datasets as tfds

def get_train_input_fn(batch_size, num_epochs, noise_dim):
    """The input function that builds the `tf.data.Dataset` object and instantiate
    the iterator correctly ready to be use.
    Returns:
        the iterator associated to the built Dataset object.
    """
    
    def convert_and_resize(features):
        image = tf.image.convert_image_dtype(features["image"], dtype=tf.float32)
        image = (image - 0.5) * 2
        image = tf.image.resize(image, size=(64, 64))
        features["image"] = image
        return features

    def _input_fn():
        real_data = tfds.load("celeb_a",split=tfds.Split.ALL)
        real_data = real_data.map(convert_and_resize).map(lambda feature: feature["image"])
        real_data = real_data.batch(batch_size, drop_remainder=True).repeat(num_epochs)
        real_data_iterator = real_data.make_one_shot_iterator()
        
        noise = tf.random_normal([batch_size, noise_dim], name="train_noise")
        real_batch = real_data_iterator.get_next()
        real_batch.set_shape((batch_size,) + tuple(real_batch.shape[1:]))
        return noise, real_batch
    return _input_fn

### Training

In [ ]:
def dcgan():
    
    # Hyperparameters
    model_dir = "../logs/"
    batch_size = 128
    num_epochs = 1
    noise_dim = 100
    
    # Run Configuration (it has other arguments)
    run_config = tf.estimator.RunConfig(
        model_dir=model_dir, save_summary_steps=50, save_checkpoints_steps=500)
    
    # Instatiate the GANEstimator object
    gan_estimator = tfgan.estimator.GANEstimator(
        config=run_config,
        generator_fn=generator_fn,
        discriminator_fn=discriminator_fn,
        generator_loss_fn=tfgan.losses.modified_generator_loss,
        discriminator_loss_fn=tfgan.losses.modified_discriminator_loss,
        generator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        discriminator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        add_summaries=tfgan.estimator.SummaryType.IMAGES
    )
    
    # Instatiate the train_input_fn
    # The model will train until it exhausts the Dataset which is repeated EPOCH times
    train_input_fn = get_train_input_fn(batch_size, num_epochs, noise_dim)
    trained_model = gan_estimator.train(train_input_fn, max_steps=None)
    return trained_model

### TensorBoard

In order to track our training we need to launch a **TensorBoard** session pointing to the folder (`model_dir`) containing the logs generated by our training.

```bash
tensorboard --logdir=PATH_TO_YOUR_MODEL_DIR
```

In [ ]:
trained_model = dcgan()

## Predictions

We train Machine Learning models because we want them to perform some specific task since we now have our GAN trained, we can finally use it to "predict" AKA generate a new image from a noise vector.

Once again, TFGAN makes it as easy as invoking the `predict()`  method of our `GANEstimator` while passing to it a `predict_input_fn` as a required argument. 

### predict_input_fn()

As previously mentioned while theoretically identical, `train_input_fn` and `predict_nput_fn()` should be implemented differently.  The first one is a simple `tf.random_normal()` node, the second should make proper use of the `tf.Dataset` API.

In [ ]:
def _predict_input_fn(batch_size, noise_dims=100, **kwargs):
    
    def predict_input_fn():
        noise_gen = np.array([np.float32(np.random.normal(size=[1, noise_dims])) for i in range(batch_size)])
        dataset = tf.data.Dataset.from_tensor_slices(noise_gen)
        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
        
    return predict_input_fn

In [ ]:
predictions_batch = 3
predict_input_fn = _predict_input_fn(batch_size=predictions_batch)
predictions = trained_model.predict(predict_input_fn)
[next(predictions) for _ in range(predictions_batch)]

## Preparing for Production

Now that we have our model trained and ready we are just a couple steps away from being able to put our model into production.

### serving_input_receiver_fn

To serve a model in production, we first need to equip it with an interface which will receive data from our requests, such interface is specified by the aptly named `serving_input_receiver_fn`. Of the three input functions, this is the trickiest one since it has its peculiar API.

This functions requires its output to be a either a `ServingInputReceiver` or a `TensorServingInputReceiver` object; the documentation on their use is clear:

> The normal `ServingInputReceiver` always returns a feature dict, even if it contains only one entry, and so can be used only with models that accept such a dict. 
>For models that accept only a single raw feature, the `serving_input_receiver_fn` provided to `Estimator.export_savedmodel()` should return this `TensorServingInputReceiver`.

Since our model needs only a noise vector to get going, we can use `TensorServingInputReceiver`.

>The expected return values are: 
> - **features**: A single `Tensor` or `SparseTensor`, representing the feature to be passed to the model. 
> - **receiver_tensors**: A Tensor, SparseTensor, or dict of string to Tensor or SparseTensor, specifying input nodes where this receiver expects to be fed by default. Typically, this is a single placeholder expecting serialized `tf.Example` protos. 
> - **receiver_tensors_alternatives**: a dict of string to additional groups of receiver tensors, each of which may be a `Tensor`, `SparseTensor`, or dict of string to `Tensor` or `SparseTensor`. These named receiver tensor alternatives generate additional serving signatures, which may be used to feed inputs at different points within the input receiver subgraph. A typical usage is to allow feeding raw feature Tensors downstream of the `tf.parse_example()` op. Defaults to None.

In [ ]:
def _serving_input_receiver_fn():
    """Instantiate a placeholder for our serving input data.

    Call the custom function `serving_input_fn()`, built following
    TensorFlow Estimator API convention, initializing the placeholder for
    the noise we will feed the model during its serving.

    The Serving Input function has two key elements:

        - the data-processing step, where we concretely prepare data to be
        fed to the:
        - Placeholder, it is the node where the input are fed.

    The things to notice is that while using `ServingInputReceiver`
    your data processing step should have at its core the parsing of the
    tf.Example received.

    With `TensorServingInputReceiver` our data won't really be passed by the
    request, instead it will have to be generated ''model-side'' inside the
    `_serving_input_receiver_fn` itself.

    Returns:
        tf.estimator.export.TensorServingInputReceiver passing the
        placeholder for the noise to it.
    """

    receiver_tensors = tf.placeholder(shape=[None, 100], dtype=tf.float32, name="serving_noise")
    return tf.estimator.export.TensorServingInputReceiver(receiver_tensors, receiver_tensors)

### Exporting the model for production

Exporting the model is as easy as calling `GANEstimator.export_savedmodel()` which the same as the normal `Estimator`.

> This method builds a new graph by first calling the serving_input_receiver_fn to obtain feature Tensors, and then calling this Estimator's model_fn to generate the model graph based on those features. It restores the given checkpoint (or, lacking that, the most recent checkpoint) into this graph in a fresh session. Finally it creates a timestamped export directory below the given export_dir_base, and writes a SavedModel into it containing a single MetaGraphDef saved from this session.

We have to specify an output folder and the `serving_input_receiver_fn`.

In [ ]:
export_dir_base = "../assets/exported_models"
trained_model.export_savedmodel(
    export_dir_base, _serving_input_receiver_fn)

### Local CloudML Predictions testing

Before going over the required steps for model serving, we want to test it locally to make sure that the exported models will behave correctly once loaded onto CloudML Engine.

#### Generate the test noise

Google Cloud often uses Newline Delimited JSON  when working with JSON-formatted data; we can use the `jsonlines` Python library assure our compliance with the standard.

In [ ]:
import jsonlines

In [ ]:
BATCH_SIZE = 2
NOISE_DIMS = 100

with jsonlines.open("../assets/test_noise.ndjson", "w") as writer:
    noise = np.random.normal(size=(BATCH_SIZE, NOISE_DIMS)).tolist()
    noised_dict = [{"input": n} for n in noise]
    writer.write_all(noised_dict)

#### Execute the test

In [ ]:
%%bash

MODEL_DIR="../assets/exported_models"
MODEL_ID="1535124551"
JSON_INSTANCES="../assets/test_noise.ndjson"

rm ~/google-cloud-sdk/lib/googlecloudsdk/command_lib/ml_engine/*.pyc

gcloud ml-engine local predict \
    --model-dir=$MODEL_DIR/$MODEL_ID \
    --json-instances=$JSON_INSTANCES

## To the serving

Now that we have the exported model we are ready to finally serve our model.

## NOTES

[1]: [The GAN Landscape: Losses, Architectures, Regularization, and Normalization](https://arxiv.org/abs/1807.04720v1), which in our opinion is one of the most thorough scientific studies on GANs out there, suggests that `Spectral Normalization` is the real big deal and that `BatchNorm` actually hurt performance if applied to the Discriminator. We still decided to go with the classic formulation of DCGAN as not to overtax you with theoretical discussions. We leave the implementation of the `Spectral Normalization`(and SNGAN) to you as an exercise. On the theoretical treating of `BatchNorm` we recommend a back to back reading of [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167), [Understanding Batch Normalization](https://arxiv.org/abs/1806.02375v1), [How Does Batch Normalization Help Optimization? (No, It Is Not About Internal Covariate Shift)](https://arxiv.org/abs/1805.11604) 

[2]: While often you want to work with **features (and labels)** fetched from a `tf.Dataset`, the **noise** should always be instantiated using a simple TensorFlow node. Trying to create a noise-containing `tf.Dataset` for the `train_input_fn` is not worth the effort. NOTE: that as things are different for the [predict_input_fn](##predict_input_fn()). 

## Links

Generative Adversarial Networks : https://arxiv.org/abs/1406.2661

really-awesome-gan : https://github.com/nightrome/really-awesome-gan

DCGAN : https://arxiv.org/abs/1511.06434

Deconvolution and Checkerboard Artifacts : https://distill.pub/2016/deconv-checkerboard/

TFGAN MNIST GAN Example : https://github.com/tensorflow/models/tree/master/research/gan/mnist_estimator

Estimator API : https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator

export_savedmodel : https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#export_savedmodel